In [1]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from cv2 import cuda

In [2]:
video_files = glob.glob('video_footages/*')
video_files

['video_footages\\Raw Video_ Pittsburg Neighborhood Drive-By Shootings.mp4',
 'video_footages\\Road traffic video for object recognition.mp4',
 'video_footages\\Shooting captured by surveillance camera in Parma.mp4',
 'video_footages\\Shopping, People, Commerce, Mall, Many, Crowd, Walking   Free Stock video footage   YouTube.mp4',
 'video_footages\\Traffic Flow In The Highway - 4K Stock Videos _ NoCopyright _ AllVideoFree.mp4',
 'video_footages\\vecteezy_busy-street-in-evening-hong-kong_28840705 (1) (1).mp4',
 'video_footages\\vecteezy_people-crossing-the-road-on-zebra-tallin_28257759.mp4']

In [10]:
gpu_flow = cv2.cuda_FarnebackOpticalFlow.create(
    5, 0.5, False, 15, 3, 5, 1.2, 0
)

AttributeError: module 'cv2' has no attribute 'cuda_FarnebackOpticalFlow'

In [6]:
def get_motion_mask(flow_mag, motion_thresh=1, kernel=np.ones((7,7))):

    motion_mask = np.uint8(flow_mag > motion_thresh)*255

    motion_mask = cv2.erode(motion_mask, kernel, iterations=1)
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_OPEN, kernel, iterations=1)
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_CLOSE, kernel, iterations=3)
    
    return motion_mask

In [7]:
cap = cv2.VideoCapture(video_files[6])

mask_kernel = np.ones((7,7), dtype=np.uint8)

thresh = 400

ret, old_frame = cap.read()
height, width, layers = old_frame.shape
new_h = height / 2
new_w = width / 2
motion_thresh = np.c_[np.linspace(0.3, 1, int(new_h))].repeat(int(new_w), axis=-1)
old_frame = cv2.resize(old_frame, (int(new_w), int(new_h)))
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(old_frame)
hsv[..., 1] = 255

while(cap.isOpened()):
    
    ret, frame = cap.read()
    if ret == True:
      
        height, width, layers = frame.shape
        new_h = height / 2
        new_w = width / 2
        frame = cv2.resize(frame, (int(new_w), int(new_h)))
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # calculate optical flow
        flow = cv2.calcOpticalFlowFarneback(old_gray, frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        # flow image output:
        hsv[..., 0] = ang*180/np.pi/2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

        # bounding boxes:
        motion_mask = get_motion_mask(mag, motion_thresh=motion_thresh, kernel=mask_kernel)

        contours, _ = cv2.findContours(motion_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_L1)

        for cnt in contours:
            x,y,w,h = cv2.boundingRect(cnt)
            area = w*h
            if area > thresh:
                frame = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), thickness = 2)
        

        #cv2.imshow('flow', draw_flow(frame_gray, flow))
        #cv2.imshow('flow HSV', draw_hsv(flow))

        cv2.imshow('motion', motion_mask)
        cv2.imshow('video', frame)
        cv2.imshow('flow', bgr)
    
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

        old_gray = frame_gray.copy()

    else:
        break

cap.release()

cv2.destroyAllWindows()

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(uint8) found for signature:
 
 >>> <unknown function>(array(bool, 2d, C))
 
There are 2 candidate implementations:
[1m - Of which 2 did not match due to:
 Overload in function 'make_callable_template.<locals>.generic': File: numba\core\typing\templates.py: Line 174.
   With argument(s): '(array(bool, 2d, C))':[0m
[1m  Rejected as the implementation raised a specific error:
    TypingError: [1mCasting array(bool, 2d, C) to uint8 directly is unsupported. Try doing '<array>.astype(np.uint8)' instead[0m[0m
  raised from C:\Users\An_asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\numba\core\typing\builtins.py:851
[0m
[0m[1mDuring: resolving callee type: class(uint8)[0m
[0m[1mDuring: typing of call at C:\Users\An_asus\AppData\Local\Temp\ipykernel_19760\1984252410.py (4)
[0m
[1m
File "..\..\..\..\AppData\Local\Temp\ipykernel_19760\1984252410.p